In [150]:
%matplotlib inline

In [155]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import pandas as pd
from scipy import stats
import numpy as np
import datetime


import pickle
from tqdm import tqdm

from ipywidgets import widgets
from IPython.display import display
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets

import geojson
import json

import folium
from folium import plugins
import folium.colormap as cmap


In [154]:
regions = pd.read_csv('regions.csv',sep = ';')

with open('dfres_c7.pickle', 'rb') as f:
    dfres_c = pickle.load(f) 
with open('preddf.pickle', 'rb') as f:
    preddf = pickle.load(f)    
with open('realdf.pickle', 'rb') as f:
    realdf = pickle.load(f) 

date_list_june = [datetime.datetime.strptime("2016-05-31 23:00", "%Y-%m-%d %H:%M") + relativedelta(hours=x) 
             for x in range(0,715)]  


100%|██████████| 715/715 [1:20:23<00:00,  6.79s/it]


### Отбражение фактических и прогнозных поездок желтого такси в карте Нью-Йорка

In [159]:
plt.figure(figsize(15,7))
list_of_features = []
for ind in regions.index:
    poly = geojson.Polygon([[[regions.iloc[ind].west, regions.iloc[ind].south], 
                             [regions.iloc[ind].west, regions.iloc[ind].north],
                             [regions.iloc[ind].east, regions.iloc[ind].north], 
                             [regions.iloc[ind].east, regions.iloc[ind].south],
                             [regions.iloc[ind].west, regions.iloc[ind].south]]])
    feature = geojson.Feature(geometry=poly, id=str(ind), properties={"name": "reg. {}".format(int(regions.iloc[ind].region))})
    list_of_features.append(feature)
features = geojson.FeatureCollection(list_of_features)

data_reg_mean = preddf[date_list_june[0]]
vmin = dfres_c.pred.min()
vmax = dfres_c.pred.max()
width = vmax - vmin
color_func = cmap.LinearColormap(['white', 'green', 'cyan', 'blue', 'magenta', 'red'],
                               vmax = vmax, vmin = vmin,
                               index=[vmin, 20, 0.3*width, 0.6*width, 0.9*width, 1.0*vmax])


map_nyc = folium.Map(location=[(nyc_latitude_south+nyc_latitude_north)/2, (nyc_longitude_east+nyc_longitude_west)/2],
                     zoom_start = 11, tiles= 'cartodbpositron')

gj = folium.GeoJson(
    features,
    style_function=lambda feature: {
        'fillColor': color_func(data_reg_mean[int(feature['id'])]),
        'color': 'black',
        'weight': 0.2,
        'dashArray': '5, 5'
    }
)
gj.add_to(map_nyc)

def f (x):
    print(x)
    #print(preddf[date_list_june[x]])
    data_reg_mean = preddf[x]
    gj.style_function = lambda feature: {
        'fillColor': color_func(data_reg_mean[int(feature['id'])]),
        'color': 'black',
        'weight': 0.2,
        'dashArray': '5, 5'
    }
    display(map_nyc)
 
def f_real (x):
    print(x)
    #print(preddf[date_list_june[x]])
    data_reg_mean = realdf[x]
    gj.style_function = lambda feature: {
        'fillColor': color_func(data_reg_mean[int(feature['id'])]),
        'color': 'black',
        'weight': 0.2,
        'dashArray': '5, 5'
    }
    display(map_nyc)
    

color_func.caption = "Mean value of trips"
map_nyc.add_child(color_func)   
interact(f, x=widgets.SelectionSlider(
    options=date_list_june,
    value=date_list_june[0],
    description=u'Прогноз',
    disabled=False,

    continuous_update=False,
    orientation='horizontal',
    readout=True,

));

interact(f_real, x=widgets.SelectionSlider(
    options=date_list_june,
    value=date_list_june[0],
    description=u'Факт',
    disabled=False,

    continuous_update=False,
    orientation='horizontal',
    readout=True,

));

### Отбражение фактических и прогнозных поездок желтого такси в Нью-Йорке в июне 2016 года по регионам

In [160]:
def f1(x):
    df1 = pd.DataFrame(dfres_c[dfres_c.region == x].values,columns=['region', 'date', 'quantity','lr_pred','pred'],index= dfres_c[dfres_c.region == 1075].date.values)
    df1.quantity.plot()
    df1.pred.plot(color='r')
    plt.ylabel(u'Количество поездок')
    plt.legend([u'Факт',u'Предсказание'])
    plt.show()
plt.figure(figsize(15,7))
    
interact(f1, x= widgets.Dropdown(
    options=np.unique(dfres_c.region.values).tolist(),
    value=1075,
    description=u'Номер региона:',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
    ))

<function __main__.f1>